# Final model

We have recreated a discrete-event simulation (DES) model designed to simulate patient activity from six different sources in a 24-bed critical care unit (CCU). This model intends to investigate the changes of elective surgery cancellations and occupied bed numbers with the increase of funded bed counts.

## Model Description

In the paper published by Griffiths et al. (2010), the simulation model admits patients from two different routes: `"unplanned admission"` and `"planned admission"`. In the former route, patients from five various sources have to wait until a bed is available, then they can enter the unit and stay there for a specific period of time to receive therapy. After the treatment is completed, they will leave the system immediately. In the latter route, elective surgery patients can only enter the system when there is a free bed, or they have to cancel the surgery if no bed is available at present. After their admission and treatment, they will leave the system immediately. A period of changeover is necessary after their discharge. The diagram of the model is displayed in Figure 1.

&nbsp;

<div style="text-align: center;"><em>Figure 1. Critical care unit simulation model diagram. IAT: inter-arrival time; LOS, length of stay..</em></div>
<img src="diagram.png" width="900" style="display: block; margin: 0 auto">


## Parameters
The input parameters that configure each simulation are:

**Inter-arrival time, IAT & daily arrivals**

Intervals of patient arrivals. For elective surgery patients, we modelled their daily arrival times.

| Source        	| Distribution 	| Mean (hours) 	| Standard Dev (hours) 	|
|-------------------|--------------	|-------------	|-----------------------|
|  A&E              | Exponential  	| 22.72         |                       |
|  Ward             | Exponential  	| 26.0          |                       |
|  Emergency        | Exponential  	| 37.0          |                       |
|  Other hospitals  | Exponential  	| 47.2          |                       |
|  X-Ray            | Exponential  	| 575.0         |                       |
|  Elective, daily arrivals         | Normal    	| 17.91         |  3.16                 |

  
**Length of stay, LOS**

The time patients spent in the CCU.

| Source        	| Distribution 	| Mean (hours) 	| Standard Dev (hours) 	|
|-------------------|--------------	|-------------	|-----------------------|
|  A&E              | Lognormal  	| 128.79        |  267.51               |
|  Ward             | Lognormal  	| 177.89        |  276.54               |
|  Emergency        | Lognormal  	| 140.15        |  218.02               |
|  Other hospitals  | Lognormal  	| 212.86        |  457.67               |
|  X-Ray            | Lognormal  	| 87.53         |  108.15               |
|  Elective         | Lognormal    	| 57.34         |  99.78                |


**Changeover time**

After each discharge, the time required for bed and the surrounding area cleaning.      

| Source        	| Distribution 	| Mean (hours) 	| Minimum (hours) 	| Maximum (hours) 	|
|-------------------|--------------	|-------------	|-----------------------|-----------------------|
|  Changeover              | Triangular  	| 5        |  2               |  8               |


## Scenario test
-   Base scenario: 24 beds
-	Changing the number of funded beds (ranging from 22 to 29)

## Outcomes
Primary outcomes include:
- Mean number of beds occupied
- Number of elective surgery cancellations
- Total admissions count

Secondary outcomes include:
- Occupancy rate
- Mean beds in queue	
- Mean waiting hours
- Admissions from six sources

### Set up environment:

First, set up a virtual conda environment to run the simulation:

> conda env create -f environment.yml

> conda activate simulation-dev

### Single run:

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from simulation import (CCU, 
                        Scenario,
                        single_run,
                        multiple_replications)

In [2]:
# create the default scenario
args = Scenario()

# single run
print('Running simulation ...', end=' => \n')
results = single_run(args,  
                    random_no_set=42)
print('simulation complete.')
results

Running simulation ... => 
simulation complete.


,A&E_admissions,Bed_days_util,Cancelled_Surgeries,Elective_Surgery_admissions,Emergency_admissions,Mean_wait_hours,Other_hospital_admissions,Total_admissions,Ward_admissions,Xray_admissions,mean_queue_bed,mean_occupied_bed,occupancy_rate
1,436.0,0.92,552.0,288.0,253.0,27.88,205.0,1579.0,383.0,14.0,4.38,23.12,0.96


### Multiple replications:

We have found the optimal replication number with the confidence interval method (n=21), as documented in the 03_iter.ipynb.

In [3]:
%%time

# default scenario
args = Scenario()


# run multiple replications (n=21).
print('Running multiple replications', end=' => \n')
results = multiple_replications(args, 
                                n_reps=21)

print('done.\n')

# show first 5 reps 
results.head()

Running multiple replications => 
done.

Wall time: 2.82 s


,A&E_admissions,Bed_days_util,Cancelled_Surgeries,Elective_Surgery_admissions,Emergency_admissions,Mean_wait_hours,Other_hospital_admissions,Total_admissions,Ward_admissions,Xray_admissions,mean_queue_bed,mean_occupied_bed,occupancy_rate
rep,,,,,,,,,,,,,
1,440.0,0.96,615.0,225.0,250.0,35.22,181.0,1458.0,347.0,15.0,8.53,23.21,0.97
2,427.0,0.93,559.0,281.0,241.0,11.10,179.0,1499.0,356.0,15.0,2.57,23.16,0.96
3,426.0,0.94,560.0,280.0,277.0,10.30,221.0,1552.0,336.0,12.0,2.70,23.27,0.97
4,395.0,0.92,517.0,323.0,255.0,8.55,204.0,1549.0,349.0,23.0,2.30,23.13,0.96
5,431.0,0.88,408.0,432.0,253.0,9.34,199.0,1642.0,313.0,14.0,2.36,22.33,0.93


In [4]:
# Display the means of metrics
results.mean().round(2)

A&E_admissions                  416.67
Bed_days_util                     0.93
Cancelled_Surgeries             535.90
Elective_Surgery_admissions     304.10
Emergency_admissions            249.57
Mean_wait_hours                  20.56
Other_hospital_admissions       194.48
Total_admissions               1536.81
Ward_admissions                 355.29
Xray_admissions                  16.71
mean_queue_bed                    4.74
mean_occupied_bed                23.03
occupancy_rate                    0.96
dtype: float64

### Comparison with real data:

In the paper, authors reported that the actual number of annual admissions was 1359, the mean number of beds occupied at any time during the year was 20.10, and the number of elective surgery cancellations was 57 when 24 beds were utilised. We will perform Student's t-test to compare the simulation results to the actual figures.

In [5]:
# data reported in the paper
total_admission = 1359
mean_occupied_beds = 20.10
es_cancellation = 57


print(f"total_admission: {stats.ttest_1samp(results['Total_admissions'].to_numpy(), total_admission)}")
print(f"mean_occupied_bed: {stats.ttest_1samp(results['mean_occupied_bed'].to_numpy(), mean_occupied_beds)}")
print(f"Cancelled_Surgeries: {stats.ttest_1samp(results['Cancelled_Surgeries'].to_numpy(), es_cancellation)}")

total_admission: TtestResult(statistic=15.343069287870206, pvalue=1.5847733993414938e-12, df=20)
mean_occupied_bed: TtestResult(statistic=42.691439645324664, pvalue=4.020110942897368e-21, df=20)
Cancelled_Surgeries: TtestResult(statistic=34.21666820844597, pvalue=3.1719282973272563e-19, df=20)


Hence, we found that the performance metrcis from our simulation model were significantly different from the actual data (all P < 0.05), indicating that while the model effectively simulated patient activity processes, it cannot fully replicate the original model due to insufficient information on statistical distribution of elective surgery patients.

### Scenario test:

We have set up several what-if scenarios by changing the number of funded beds.

In [6]:
# setup scenarios
def get_scenarios():
    # store scenarios in a dict
    scenarios = {}
    
    # 22 beds
    scenarios['22_beds'] = Scenario()
    scenarios['22_beds'].n_beds -= 2
    
    # 23 beds
    scenarios['23_beds'] = Scenario()
    scenarios['23_beds'].n_beds -= 1
    
    # default scenario
    scenarios['24_beds'] = Scenario()
    scenarios['24_beds'].name = '24_beds'

    # 25 beds
    scenarios['25_beds'] = Scenario()
    scenarios['25_beds'].n_beds += 1
        
    # 26 beds
    scenarios['26_beds'] = Scenario()
    scenarios['26_beds'].n_beds += 2
    
    # 27 beds
    scenarios['27_beds'] = Scenario()
    scenarios['27_beds'].n_beds += 3
    
    # 28 beds
    scenarios['28_beds'] = Scenario()
    scenarios['28_beds'].n_beds += 4
    
    # 29 beds
    scenarios['29_beds'] = Scenario()
    scenarios['29_beds'].n_beds += 5  
    
        
    return scenarios

In [7]:
def run_scenario_analysis(scenarios, n_reps):
    '''
    Run each of the scenarios for a specified results
    collection period, warmup and replications.
    
    Params:
    ---------------------
    scenarios: dict 
        various scenarios from the get_scenarios function

    n_reps: int
        replication number
    '''
    print('Scenario Analysis')
    print(f'No. Scenario: {len(scenarios)}')
    print(f'Replicatins: {n_reps}')
    
    
    scenario_results = {}
    for sc_name, scenario in scenarios.items():
        
        print(f'Running {sc_name}', end=' => ')
        replications  = multiple_replications(scenario, 
                                              n_reps=n_reps)
        print('done.\n')
        
        #save the results
        scenario_results[sc_name] = replications
    
    print('Scenario analysis complete.')
    return scenario_results

In [8]:
%%time
# number of replications
N_REPS = 21

# get the scenarios
scenarios = get_scenarios()

# run the scenario analysis
scenario_results = run_scenario_analysis(scenarios, 
                                         N_REPS)

Scenario Analysis
No. Scenario: 8
Replicatins: 21
Running 22_beds => done.

Running 23_beds => done.

Running 24_beds => done.

Running 25_beds => done.

Running 26_beds => done.

Running 27_beds => done.

Running 28_beds => done.

Running 29_beds => done.

Scenario analysis complete.
Wall time: 6.75 s


In [9]:
def scenario_summary_frame(scenario_results):
    '''
    Mean results for each performance measure by scenario
    
    Parameters:
    ----------
    scenario_results: dict
        dictionary of replications.  
        Key identifies the performance measure
        
    Returns:
    -------
    pd.DataFrame
    '''
    columns = []
    summary = pd.DataFrame()
    for sc_name, replications in scenario_results.items():
        summary = pd.concat([summary, replications.mean()], axis=1)
        columns.append(sc_name)

    summary.columns = columns
    return summary

In [11]:
# display results of scenarios test
summary_frame = scenario_summary_frame(scenario_results)
summary_frame.round(2)

,22_beds,23_beds,24_beds,25_beds,26_beds,27_beds,28_beds,29_beds
A&E_admissions,416.67,416.67,416.67,416.67,416.67,416.67,416.67,416.67
Bed_days_util,0.97,0.95,0.93,0.92,0.90,0.88,0.86,0.85
Cancelled_Surgeries,689.00,612.52,535.90,457.33,388.00,319.57,263.29,207.67
Elective_Surgery_admissions,151.00,227.48,304.10,382.67,452.00,520.43,576.71,632.33
Emergency_admissions,249.57,249.57,249.57,249.57,249.57,249.57,249.57,249.57
Mean_wait_hours,56.72,33.29,20.56,12.42,8.03,5.18,3.43,2.34
Other_hospital_admissions,194.48,194.48,194.48,194.48,194.48,194.48,194.48,194.48
Total_admissions,1383.71,1460.19,1536.81,1615.38,1684.71,1753.14,1809.43,1865.05
Ward_admissions,355.29,355.29,355.29,355.29,355.29,355.29,355.29,355.29
Xray_admissions,16.71,16.71,16.71,16.71,16.71,16.71,16.71,16.71


Resutls showed that increasing the number of funded beds resulted in higher average occupied bed counts and total admissions, but led to reductions in elective surgery cancellations and occupancy rate.

# End